# Analyze Industrial composition of municipalities in the region


In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import calendar
import folium
import geopandas as gpd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import math
import folium
import datetime
from os import listdir
from os.path import isfile, join

from matplotlib import gridspec
from matplotlib.backends.backend_pdf import PdfPages

import statsmodels.api as sm
from stargazer.stargazer import Stargazer

## Read ateco 1 and ateco 2 codes and merge with their respective descriptions

In [3]:
ateco2007 = pd.read_csv('quake_data/Ateco2007.csv', sep=';')
ateco1 = ateco2007[ateco2007['Codice Ateco 2007'].str.len()==1].reset_index().drop(['index','Codice Ateco 2007'],axis=1)
ateco1.columns = ['ateco1','desc1']
ateco2 = ateco2007[ateco2007['Codice Ateco 2007'].str.len()==2].reset_index().drop('index',axis=1)
ateco2.columns = ['ateco1','ateco2','desc2']
ateco = pd.merge(ateco1,ateco2)
ateco

,ateco1,desc1,ateco2,desc2
0,A,"AGRICOLTURA, SILVICOLTURA E PESCA",01,COLTIVAZIONI AGRICOLE E PRODUZIONE DI PRODOTTI...
1,A,"AGRICOLTURA, SILVICOLTURA E PESCA",02,SILVICOLTURA ED UTILIZZO DI AREE FORESTALI
2,A,"AGRICOLTURA, SILVICOLTURA E PESCA",03,PESCA E ACQUACOLTURA
3,B,ESTRAZIONE DI MINERALI DA CAVE E MINIERE,05,ESTRAZIONE DI CARBONE (ESCLUSA TORBA)
4,B,ESTRAZIONE DI MINERALI DA CAVE E MINIERE,06,ESTRAZIONE DI PETROLIO GREGGIO E DI GAS NATURALE
...,...,...,...,...
83,S,ALTRE ATTIVITÀ DI SERVIZI,95,RIPARAZIONE DI COMPUTER E DI BENI PER USO PERS...
84,S,ALTRE ATTIVITÀ DI SERVIZI,96,ALTRE ATTIVITÀ DI SERVIZI PER LA PERSONA
85,T,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
86,T,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,98,PRODUZIONE DI BENI E SERVIZI INDIFFERENZIATI P...


In [4]:
# aida_crateri3.csv contains companies that were founded before the earthquake (Aug. 2016) and were active when the earthquake hit

aida = pd.read_csv('quake_data/aida_crateri3.csv')
aida = aida[['Comune','Ricavi delle vendite migl EUR Ultimo anno disp.','Dipendenti Ultimo anno disp.','Stato giuridico','ateco2']]
aida.columns = ['Comune','ricavi','dipendenti','stato','ateco2']

aida['ateco2'] = aida['ateco2'].astype('string')
m = aida['ateco2'].str.len().max()
aida['ateco2'] = aida['ateco2'].str.rjust(m, '0').astype('str').str[0:2]

aida[aida['ateco2']=='02']

,Comune,ricavi,dipendenti,stato,ateco2
115,Fabriano,2010.0,3,Attiva,02
183,Fabriano,601.0,0,Attiva,02
190,Fabriano,537.0,42,Fallita,02
249,Fabriano,250.0,30,Cessata (Fusione),02
434,Fabriano,1.0,2,Attiva,02
...,...,...,...,...,...
52484,Fara in Sabina,160.0,3,Attiva,02
52750,Caramanico Terme,136.0,1,Attiva,02
52911,Magliano de' Marsi,132.0,5,Attiva,02
53167,Massa d'Albe,0.0,0,Attiva,02


In [5]:
aida_ateco = pd.merge(aida.astype('str') , ateco.astype('str') , how='left', on='ateco2')
aida_ateco.loc[aida_ateco['ricavi'] == 'n.d.','ricavi'] = np.nan
aida_ateco.loc[aida_ateco['dipendenti'] == 'n.d.','dipendenti'] = np.nan
aida_ateco['ricavi'] = aida_ateco['ricavi'].astype('float')
aida_ateco['dipendenti'] = aida_ateco['dipendenti'].astype('float')
aida_ateco[aida_ateco['ateco1']=='A']
aida_ateco.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53289 entries, 0 to 53288
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Comune      53289 non-null  object 
 1   ricavi      53289 non-null  float64
 2   dipendenti  53289 non-null  float64
 3   stato       53289 non-null  object 
 4   ateco2      53289 non-null  object 
 5   ateco1      53289 non-null  object 
 6   desc1       53289 non-null  object 
 7   desc2       53289 non-null  object 
dtypes: float64(2), object(6)
memory usage: 3.7+ MB


In [6]:
aida_ateco.loc[aida_ateco['Comune']=='Acciano',:]

,Comune,ricavi,dipendenti,stato,ateco2,ateco1,desc1,desc2
52738,Acciano,120.0,2.0,Attiva,46,G,COMMERCIO ALL'INGROSSO E AL DETTAGLIO; RIPARAZ...,COMMERCIO ALL'INGROSSO (ESCLUSO QUELLO DI AUTO...
52739,Acciano,38.0,1.0,Attiva,81,N,"NOLEGGIO, AGENZIE DI VIAGGIO, SERVIZI DI SUPPO...",ATTIVITÀ DI SERVIZI PER EDIFICI E PAESAGGIO
52740,Acciano,8.0,0.0,Attiva,46,G,COMMERCIO ALL'INGROSSO E AL DETTAGLIO; RIPARAZ...,COMMERCIO ALL'INGROSSO (ESCLUSO QUELLO DI AUTO...


**aida_ateco** dataframe contains all the companies in the region that were active when the earthquake hit, together with their ateco codes

In [7]:
# Ateco 1  sectors and some of their statistics
vc = aida_ateco['desc1'].value_counts().to_frame()
vc['p'] = vc['desc1']/vc['desc1'].sum()*100
vc

,desc1,p
COMMERCIO ALL'INGROSSO E AL DETTAGLIO; RIPARAZIONE DI AUTOVEICOLI E MOTOCICLI,9852,18.487868
ATTIVITÀ MANIFATTURIERE,9660,18.127569
COSTRUZIONI,9443,17.720355
ATTIVITA' IMMOBILIARI,6251,11.730376
"ATTIVITÀ PROFESSIONALI, SCIENTIFICHE E TECNICHE",3505,6.577342
ATTIVITÀ DEI SERVIZI DI ALLOGGIO E DI RISTORAZIONE,3074,5.768545
"NOLEGGIO, AGENZIE DI VIAGGIO, SERVIZI DI SUPPORTO ALLE IMPRESE",2197,4.122802
SERVIZI DI INFORMAZIONE E COMUNICAZIONE,2085,3.912627
TRASPORTO E MAGAZZINAGGIO,1594,2.991236
"AGRICOLTURA, SILVICOLTURA E PESCA",1403,2.632814


In [8]:
# We condier only the firts 10 sectors for the main analysis (smaller sectors are less significant)
sectors = aida_ateco['desc1'].value_counts()[0:10].index
aida_ateco = aida_ateco.loc[aida_ateco['desc1'].isin(sectors),:]

In [9]:

def ateco_groupby(df):
    aggregation = {'Comune':[np.size],'ricavi':[np.sum],'dipendenti':[np.sum]}
    ateco0 = df.groupby(['Comune']).aggregate(aggregation).reset_index()
    ateco0.columns = ['Comune','n_imprese','tot_ricavi','tot_dipendenti']
    ateco1 = df.groupby(['Comune', 'ateco1','desc1']).aggregate(aggregation).reset_index()
    ateco1.columns = ['Comune','ateco1','desc1','n_imprese','tot_ricavi','tot_dipendenti']
    ateco2 = df.groupby(['Comune','ateco2','desc2']).aggregate(aggregation).reset_index()
    ateco2.columns = ['Comune','ateco2','desc2','n_imprese','tot_ricavi','tot_dipendenti']
    return (ateco0,ateco1,ateco2)

imprese_ateco0,imprese_ateco1,imprese_ateco2 = ateco_groupby(aida_ateco)

imprese_ateco0.loc[imprese_ateco0['Comune']=='Acciano',:]

,Comune,n_imprese,tot_ricavi,tot_dipendenti
1,Acciano,3,166.0,3.0


In [10]:
imprese_ateco1.loc[imprese_ateco1['Comune']=='Acciano',:]

,Comune,ateco1,desc1,n_imprese,tot_ricavi,tot_dipendenti
2,Acciano,G,COMMERCIO ALL'INGROSSO E AL DETTAGLIO; RIPARAZ...,2,128.0,2.0
3,Acciano,N,"NOLEGGIO, AGENZIE DI VIAGGIO, SERVIZI DI SUPPO...",1,38.0,1.0


In [11]:
imprese_ateco2.loc[imprese_ateco2['Comune']=='Acciano',:]

,Comune,ateco2,desc2,n_imprese,tot_ricavi,tot_dipendenti
2,Acciano,46,COMMERCIO ALL'INGROSSO (ESCLUSO QUELLO DI AUTO...,2,128.0,2.0
3,Acciano,81,ATTIVITÀ DI SERVIZI PER EDIFICI E PAESAGGIO,1,38.0,1.0


**imprese_ateco1** and **imprese_ateco2** are the group_by of the companies by ateco1 and ateco2 respectively. 

In [12]:
# For each municipality we compute it industrial composition in terms of ateco codes. 
# The result will be something like: municipality X' companies are about: 50% agricolture, 40% industrial, etc.


denom = imprese_ateco0[['Comune','n_imprese','tot_ricavi']]
denom.columns = ['Comune','n_imprese0','tot_ricavi0']

imprese_ateco2 = imprese_ateco2.merge(denom)
imprese_ateco2['p_imprese'] = imprese_ateco2['n_imprese'] / imprese_ateco2['n_imprese0']
imprese_ateco2['p_ricavi'] = imprese_ateco2['tot_ricavi'] / imprese_ateco2['tot_ricavi0']

imprese_ateco1 = imprese_ateco1.merge(denom)
imprese_ateco1['p_imprese'] = imprese_ateco1['n_imprese'] / imprese_ateco1['n_imprese0']
imprese_ateco1['p_ricavi'] = imprese_ateco1['tot_ricavi'] / imprese_ateco1['tot_ricavi0']
imprese_ateco1.loc[imprese_ateco1['Comune']=='Acciano',:]

,Comune,ateco1,desc1,n_imprese,tot_ricavi,tot_dipendenti,n_imprese0,tot_ricavi0,p_imprese,p_ricavi
2,Acciano,G,COMMERCIO ALL'INGROSSO E AL DETTAGLIO; RIPARAZ...,2,128.0,2.0,3,166.0,0.666667,0.771084
3,Acciano,N,"NOLEGGIO, AGENZIE DI VIAGGIO, SERVIZI DI SUPPO...",1,38.0,1.0,3,166.0,0.333333,0.228916


In [13]:
value = 'p_ricavi'
imprese_ateco_wide = imprese_ateco1.pivot(index='Comune',columns='desc1')[value].fillna(0)
imprese_ateco_wide.columns

Index(['AGRICOLTURA, SILVICOLTURA E PESCA', 'ATTIVITA' IMMOBILIARI',
       'ATTIVITÀ DEI SERVIZI DI ALLOGGIO E DI RISTORAZIONE',
       'ATTIVITÀ MANIFATTURIERE',
       'ATTIVITÀ PROFESSIONALI, SCIENTIFICHE E TECNICHE',
       'COMMERCIO ALL'INGROSSO E AL DETTAGLIO; RIPARAZIONE DI AUTOVEICOLI E MOTOCICLI',
       'COSTRUZIONI',
       'NOLEGGIO, AGENZIE DI VIAGGIO, SERVIZI DI SUPPORTO ALLE IMPRESE',
       'SERVIZI DI INFORMAZIONE E COMUNICAZIONE', 'TRASPORTO E MAGAZZINAGGIO'],
      dtype='object', name='desc1')

In [14]:
value = 'p_ricavi'
imprese_ateco_wide = imprese_ateco1.pivot(index='Comune',columns='desc1')[value].fillna(0)
#imprese_ateco_wide = imprese_ateco2.pivot(index='Comune',columns='ateco2')[value].fillna(0)


ateco_names = ['AGRICOLTURE','REAL ESTATE','HOSPITALITY','MANUFACTURING','TECH SERVICES','WHOLE/RETAIL SALES','CONSTRUCTIONS','BUSINESS SUPPORT SERVICES','COMMUNICATION AND INFORMATION SERV.','LOGISTICS']

'''
ateco_names = ['AGRICOLTURE', 'OTHER','PUBLIC ADMINISTRATION','REAL ESTATE',
       'SPORT AND ENTERTAINMENT','HOSPITALITY',
       'FINANCIAL SERVICES', 'MANUFACTURING','TECH SERVICES',
       'WHOLE/RETAIL SALES','CONSTRUCTIONS', 'MINING',
       'WATER AND WASTE DISPOSAL','UTILITIES',
       'EDUCATION','BUSINESS SUPPORT SERVICES',
       'EXTRATERRITORIAL ORG.','SOCIAL WELFARE',
       'COMMUNICATION AND INFORMATION SERV.','LOGISTICS']
'''


imprese_ateco_wide.columns = ateco_names
imprese_ateco_wide = imprese_ateco_wide.reset_index()
imprese_ateco_wide.head(2)

,Comune,AGRICOLTURE,REAL ESTATE,HOSPITALITY,MANUFACTURING,TECH SERVICES,WHOLE/RETAIL SALES,CONSTRUCTIONS,BUSINESS SUPPORT SERVICES,COMMUNICATION AND INFORMATION SERV.,LOGISTICS
0,Abbateggio,0.0,0.0,0.24878,0.0,0.0,0.000000,0.75122,0.000000,0.0,0.0
1,Acciano,0.0,0.0,0.00000,0.0,0.0,0.771084,0.00000,0.228916,0.0,0.0


In [15]:
imprese_ateco_wide.to_csv('quake_data/imprese_ateco_wide.csv',index=False)

The result is a file with all the municipalities and their industrial composition (percentages are in terms of the fraction of the total revenue of the sector VS. total overall revenues)